In [6]:
import requests
import json
import time
import random
from typing import List
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, StateGraph, MessagesState
from env import API_KEY


class AgentState(MessagesState):
    agent_name: str
    other_agent_name: str

def create_agent_state(name: str, other_name: str) -> AgentState:
    return AgentState(messages=[], agent_name=name, other_agent_name=other_name)

class ClaudeAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.api_url = "https://api.anthropic.com/v1/messages"
        self.headers = {
            "Content-Type": "application/json",
            "X-API-Key": self.api_key,
            "anthropic-version": "2023-06-01"
        }

    def generate_response(self, prompt):
        payload = {
            "model": "claude-3-haiku-20240307",
            "max_tokens": 2048,
            "messages": [{"role": "user", "content": prompt}]
        }
        response = requests.post(self.api_url, headers=self.headers, json=payload)
        if response.status_code == 200:
            return response.json()['content'][0]['text']
        else:
            raise Exception(f"API request failed: {response.status_code}, {response.text}")

# Claude APIのインスタンスを作成
claude_api = ClaudeAPI(API_KEY)

def generate_response_with_retry(state: AgentState, max_retries=3):
    for attempt in range(max_retries):
        try:
            messages = state['messages']
            prompt = f"あなたは{state['agent_name']}です。{state['other_agent_name']}と会話をしています。以下の会話を続けてください：\n\n"
            for message in messages:
                if isinstance(message, HumanMessage):
                    prompt += f"{state['other_agent_name']}: {message.content}\n"
                elif isinstance(message, AIMessage):
                    prompt += f"{state['agent_name']}: {message.content}\n"
            
            time.sleep(1)  # リクエスト間に遅延を追加
            response = claude_api.generate_response(prompt)
            return {"messages": [AIMessage(content=response)]}
        except Exception as e:
            if "rate_limit_error" in str(e) and attempt < max_retries - 1:
                wait_time = (2 ** attempt) + random.random()  # 指数バックオフ + ジッター
                print(f"Rate limit reached. Waiting for {wait_time:.2f} seconds before retry.")
                time.sleep(wait_time)
            else:
                raise

def create_conversation_graph(agent1_name: str, agent2_name: str):
    workflow = StateGraph(AgentState)
    
    workflow.add_node("agent1", lambda x: generate_response_with_retry(x))
    workflow.add_node("agent2", lambda x: generate_response_with_retry(x))
    
    workflow.set_entry_point("agent1")
    
    workflow.add_edge("agent1", "agent2")
    workflow.add_edge("agent2", "agent1")
    
    return workflow.compile()

# 会話グラフの作成
conversation = create_conversation_graph("本", "コップ")

def run_conversation(turns: int):
    alice_state = create_agent_state("本", "コップ")
    bob_state = create_agent_state("コップ", "本")
    
    alice_state['messages'].append(HumanMessage(content="僕はこの主人の本です。いろいろな知識を持っているんだ。コップくんは今何を考えているの？"))
    
    for _ in range(turns):
        alice_result = conversation.invoke(alice_state)
        alice_state['messages'].extend(alice_result['messages'])
        bob_state['messages'].extend(alice_result['messages'])
        
        print(f"Alice: {alice_result['messages'][-1].content}")
        
        # リクエスト間に遅延を追加
        time.sleep(1)
        
        bob_result = conversation.invoke(bob_state)
        bob_state['messages'].extend(bob_result['messages'])
        alice_state['messages'].extend(bob_result['messages'])
        
        print(f"Bob: {bob_result['messages'][-1].content}")
        print()
        
        # リクエスト間に遅延を追加
        time.sleep(1)

# 会話の実行
run_conversation(5)  # 5ターンの会話を実行

Rate limit reached. Waiting for 1.80 seconds before retry.
Rate limit reached. Waiting for 2.78 seconds before retry.


Exception: API request failed: 429, {"type":"error","error":{"type":"rate_limit_error","message":"Number of request tokens has exceeded your per-minute rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}